In [18]:
import torch
from torch import nn
from torch import optim

from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:



training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [ ]:
# USING TORCH --> NEED DEVICE 
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [20]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # required preprocessing of image data
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # first fully connected layer, maps input (28*28 pixels = 784 pixels) to 512 hidden units
            nn.ReLU(), # activation function
            nn.Linear(512, 512), # second fully connected layer, maps 512 hidden units to 512 hidden units
            nn.ReLU(), # activation function
            nn.Linear(512, 10), # third fully connected layer, maps 512 hidden units to 10 output units
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
# MOVE MODEL TO DEVICE
model = NeuralNetwork().to(device)

In [ ]:
# Define any necessary transformations
transform = transforms.Compose([
    transforms.ToTensor()
])

# CHOOSE DATASET
# MNIST (1x28x28)
train_dataset = datasets.MNIST(root="data", train=True, transform=transform, download=True) # train dataset MNIST
test_dataset = datasets.MNIST(root="data", train=False, transform=transform, download=True) # validation dataset MNIST

# # FashionMNIST (1x28x28) batch size = 64
# train_loader = datasets.FashionMNIST(train_dataset, batch_size=64, shuffle=True) # train dataloader
# test_loader = datasets.FashionMNIST(test_dataset, batch_size=64, shuffle=True) # validation dataloader

# # CIFAR-10  (3x32x32) batxh size = 64
# train_loader = datasets.CIFAR10(train_dataset, batch_size=64, shuffle=True) # train dataloader
# test_loader = datasets.CIFAR10(test_dataset, batch_size=64, shuffle=True) # validation dataloader

In [ ]:
# PICK BATCH SIZE. USE DATALOADER TO CREATE BATCHES
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)


In [ ]:
# CHOOSE LOSS FUNCTION
# CHOOSE OPTIMIZER FUNCTION
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3) # LATER --> CH0OSE LEARING RATE

In [22]:
# training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
# testing function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [24]:
# Epoch Log / Training / Testing
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300029  [   32/60000]
loss: 2.305321  [ 3232/60000]
loss: 2.295354  [ 6432/60000]
loss: 2.290295  [ 9632/60000]
loss: 2.282018  [12832/60000]
loss: 2.275435  [16032/60000]
loss: 2.264084  [19232/60000]
loss: 2.256338  [22432/60000]
loss: 2.270914  [25632/60000]
loss: 2.255828  [28832/60000]
loss: 2.260906  [32032/60000]
loss: 2.241158  [35232/60000]
loss: 2.230987  [38432/60000]
loss: 2.234001  [41632/60000]
loss: 2.222965  [44832/60000]
loss: 2.214988  [48032/60000]
loss: 2.205625  [51232/60000]
loss: 2.203035  [54432/60000]
loss: 2.206513  [57632/60000]
Test Error: 
 Accuracy: 53.4%, Avg loss: 2.189513 

Epoch 2
-------------------------------
loss: 2.199641  [   32/60000]
loss: 2.182333  [ 3232/60000]
loss: 2.200078  [ 6432/60000]
loss: 2.174893  [ 9632/60000]
loss: 2.147201  [12832/60000]
loss: 2.169989  [16032/60000]
loss: 2.080072  [19232/60000]
loss: 2.116182  [22432/60000]
loss: 2.122827  [25632/60000]
loss: 2.076064  [28832/60000